## Lab 11 - Stable Coin (DAI)

### Setup

1. Create the project directory `lab-11`.

2. Install packages.

    ```sh
    npm i ethers dotenv
    ```

3. Copy the `.env` file containing your Rinkeby mnemonic and API URL in the `lab-11` project directory.

4. Create a `scripts` directory in the `lab-11` project directory.

### Find DAI contract address on Goerli

1. Find DAI goerli address at https://chainlog.makerdao.com/

    - click on `show` button beside `MCD_DAI`. The contract address should be `0x11fE4B6AE13d2a6055C8D9cF65c55bac32B5d844`.

2. Open the DAI contract in Etherscan on Rinkeby network.

    - Go to https://goerli.etherscan.io/address/0x11fE4B6AE13d2a6055C8D9cF65c55bac32B5d844
    - Click on `Contract` tab and click on `Code`.
      If you browse at the contract's source code, you will notice all the state variables and functions that are familiar to you now when you have created your own ERC20 token in the earlier session. Notice that DAI contract was not derived from any IERC20 interface like what you have done earlier but that is fine as long as the public/external function signature and state variables remains the same as ERC20 standard.

### Find WETH contract address on Goerli

1. If you would like to trade ERC20 with ETH trade on Uniswap, you will need to use a ERC20 wrapper for ETH called WETH. To find this address:

    - Go to https://docs.uniswap.org/protocol/reference/deployments
    - Look under **Wrapped Native Token Addresses** section.
    - Get the WETH address for Rinkeby, which should be `0xc778417E063141139Fce010982780140Aa0cD5Ab`

### Find Uniswap Router address on Goerli

1. Go to the Uniswap website and look for version 2 of the router address on Rinkeby https://docs.uniswap.org/protocol/V2/reference/smart-contracts/router-02

2. The address should be `0x7a250d5630B4cF539739dF2C5dAcb4c659F2488D`

---

### Lab 10a - Create a JS script to buy DAI using ETH.

Before we can create a pool using DAI, you will first need to own DAI. You can buy DAI on any exchange, but in this session we will learn to buy DAI using ETH from Uniswap. Buy DAI is basically a swap between DAI ERC20 token with the WETH ERC20 token. This is similar to the process which you have learnt in lab 9 on the development network.

1. Create a script `buy-dai.js` in scripts directory.

    Note that you are importing `ethers` from the `ethers` and not `hardhat`.
    You also need to import and configure `dotenv`. In previous lab, `dotenv` was loaded from hardhat.config.js.

    ```js
    const ethers = require("ethers");
    require("dotenv").config();

    const UNISWAP_ROUTER_ADDRESS = "0x7a250d5630B4cF539739dF2C5dAcb4c659F2488D";
    const DAI_ADDRESS = "0x6A9865aDE2B6207dAAC49f8bCba9705dEB0B0e6D";
    const WETH_ADDRESS = "0xc778417E063141139Fce010982780140Aa0cD5Ab";
    ```

2. Load your account.

    When using Hardhat, the provider and the passphrase were configured for you when you call `ethers.getSigner()`. In this case, you will need to configure the provider yourself, using the Rinkeby API URL. You will also need to create a wallet with your mnemonic passphrase.

    ```js
    (async ()=> {
        // Load account
        const provider = new ethers.providers.JsonRpcProvider(
            process.env.RINKEBY_APIURLKEY
        );
        const account = new ethers.Wallet(
            ethers.Wallet.fromMnemonic(process.env.RINKEBY_MNEMONIC),
            provider
        );
    ```

3. Load the Uniswap Router contract.

    #### Creating contract instance with ethers.js

    - When using Hardhat, you can use the plugin `ethers.getContractAt(contractName,contractAddress)` to create an instance of the contract. To create a contract instance using ethers.js, you use the Contract constructor `new ethers.Contract(contractAddress,contractAbi,signer)`.

    - To use the constructor, you will need to provide the contract's ABI on your own. You can load the ABI file for UniswapV2Router02 from `/lab-9/artifacts/contracts/v2-periphery/UniswapV2Router02.sol/UniswapV2Router02.json` into an object and pass it to the Contract constructor.

        Example:

        ```
        const abiFile = require('...');
        const contract = new ethers.Contract('0x...',abiFile.abi,signer);
        ```

    - Alternatively, you can use a `contract fragment`. The contract fragment is array containing the function signature of the contract. This makes the code more human readable than using the ABI.

    In this case, we will be using **swapExactETHForTokens** of the UniswapV2Router02 contract. You can get the function signature from the `UniswapV2Router02.sol` source code.

    ```js
    const uniswap = new ethers.Contract(
        UNISWAP_ROUTER_ADDRESS,
        [
            "function swapExactETHForTokens(uint, address[], address, uint) payable",
        ],
        account
    );
    ```

4. Buy 0.001 ETH worth of DAI at market price

    In lab 9, you use the uniswap's **swapExactTokenForTokens()** function to trade tokenA and tokenB but in this case you are paying exact amount of ETH for certain amount of DAI. Therefore you will need to use **swapExactETHForTokens()** instead. The main difference is that instead of approving the withdrawal the contract to withdraw ETH, you need to send 0.001 ETH together with the transaction the transaction `{value:ethers.utils.parseEther('0.001)}`.

    ```js
    // Buy 0.001 ETH worth of DAI at market price
    const ts = (await ethers.provider.getBlock()).timestamp + 10 * 60 * 1000;
    const response = await uniswap.swapExactETHForTokens(
        0,
        [WETH_ADDRESS, DAI_ADDRESS],
        account.address,
        ts,
        {
            value: ethers.utils.parseEther("0.001"),
        }
    );
    const receipt = await response.wait();
    console.log(receipt);
    ```

5. Create DAI contract instance and check DAI balance.

    To check the DAI balance, you will need call the **balanceOf()** function. Using the same method above, create the DAI contract instance using the contract fragment. Then call the balanceOf function.

    ```js
        // Check DAI balance
        const dai = new ethers.Contract(
            DAI_ADDRESS,
            ["function balanceOf(address) view returns(uint)"],
            account
        );
        const balance = await dai.balanceOf(account.address);
        console.log("DAI: balance=", balance);
    })();
    ```

6. Run the script in Rinkeby.

    Make sure you have at least 0.002 ETH to complete the trade.

    ```sh
    node scripts/buy-dai.js
    ```

7. Check your metamask wallet.

    - Open metamask, and make sure you are connected to the network `Rinkeby` then click on the `Import tokens` link.

    - In the `Import Tokens` window, enter the following:

        Token Contract Address: 0x6A9865aDE2B6207dAAC49f8bCba9705dEB0B0e6D
        Token Token Symbol: DAI

    - Click on Add Custom Token.

    - Your DAI balance should show up in your wallet and it should show the amount of DAI you have purchased using 0.01 ETH.

---

### Lab 10b - Create a JS script to add a liquidity pool using DAI and TUT token.

In this session, you will use the DAI purchased from Lab10a to create a pool using your TUT tokens. Note: Ensure you have at least 10_000 DAI and 10_000 TUT in your Rinkeby wallet.

1.  Create the `add-liquidity.js` script in `scripts` directory.

    ```js
    const { ethers } = require("ethers");
    require("dotenv").config();

    const UNISWAP_ROUTER_ADDRESS = "0x7a250d5630B4cF539739dF2C5dAcb4c659F2488D";
    const DAI_ADDRESS = "0x6A9865aDE2B6207dAAC49f8bCba9705dEB0B0e6D";
    const TUT_ADDRESS = "0xf4da4f86D35CFA69f56c5cc7f357Ad03634d7a0F"; // from lab 6

    (async () => {
        // Load Account
        const provider = new ethers.providers.JsonRpcProvider(
            process.env.RINKEBY_APIURLKEY
        );
        const account = new ethers.Wallet(
            ethers.Wallet.fromMnemonic(process.env.RINKEBY_MNEMONIC),
            provider
        );

    ```

2.  Approve the router contract to withdraw 10_000 DAI and 10_000 TUT from your wallet.

    ```js
    // Approve Router to withdraw 10,000 DAI and 10,000 TUT
    const dai = new ethers.Contract(
        DAI_ADDRESS,
        ["function approve(address,uint)"],
        account
    );
    const tut = new ethers.Contract(
        TUT_ADDRESS,
        ["function approve(address,uint)"],
        account
    );
    await dai.approve(UNISWAP_ROUTER_ADDRESS, 10_000);
    await tut.approve(UNISWAP_ROUTER_ADDRESS, 10_000);
    ```

3.  Fund the pool

    ```js
    // Fund the pool without considering slippage, ie. amountMinA =0 and amountMinB=0
    const uniswap = new ethers.Contract(
            UNISWAP_ROUTER_ADDRESS,
            [
                "function addLiquidity(address,address,uint,uint,uint,uint,address,uint) returns (uint, uint, uint)",
                "function quote(uint,uint,uint) view returns(uint)",
            ],
            account
        );
        const ts = (await provider.getBlock()).timestamp + 10 * 60 * 1000;
        let response;
        response = await uniswap.addLiquidity(
            dai.address,
            tut.address,
            10_000,
            10_000,
            0,
            0,
            account.address,
            ts,
            {
                gasLimit: 2_700_000,
                gasPrice: 8_000_000_000,
            }
        );
        const receipt = await response.wait();
        console.log(receipt);
    })();
    ```

4.  Run the script

    ```sh
    node scripts/add-liquidity.js
    ```

### Lab 10c - Check balance

After funding the DAI/TUT pool successfully, you will receive Liquidity Token. Check the balance of Liquidity Tokens and the reserve in the pool.

1. Create the file `check-balance.js` in `scripts` directory.

    ```js
    const ethers = require("ethers");
    require("dotenv").config();
    const DAI_ADDRESS = "0x6A9865aDE2B6207dAAC49f8bCba9705dEB0B0e6D";
    const UNISWAP_FACTORY_ADDRESS = "0x5C69bEe701ef814a2B6a3EDD4B1652CB9cc5aA6f";
    const TUT_ADDRESS = "0xf4da4f86D35CFA69f56c5cc7f357Ad03634d7a0F"; // from lab 6

    (async () => {
        // Load Account
        const provider = new ethers.providers.JsonRpcProvider(
            process.env.RINKEBY_APIURLKEY
        );
        const account = new ethers.Wallet(
            ethers.Wallet.fromMnemonic(process.env.RINKEBY_MNEMONIC),
            provider
        );

    ```

2. Load DAI contract and check DAI balance

    ```js
    // Check DAI Balance
    const dai = new ethers.Contract(
        DAI_ADDRESS,
        ["function balanceOf(address) view returns(uint)"],
        account
    );
    const daiBalance = await dai.balanceOf(account.address);
    console.log("Account: DAI=", daiBalance.toString());
    ```

3. Load TUT contract and check TUT balance

    ```js
    // Check TUT Balance
    const tut = new ethers.Contract(
        TUT_ADDRESS,
        ["function balanceOf(address) view returns(uint)"],
        account
    );
    const tutBalance = await tut.balanceOf(account.address);
    console.log("Account: TUT=", tutBalance.toString());
    ```

4. Load Pool contract.

    ```js
    // Load Pool Contract
    const factory = new ethers.Contract(
        UNISWAP_FACTORY_ADDRESS,
        ["function getPair(address,address) view returns(address)"],
        account
    );
    const poolAddress = await factory.getPair(dai.address, tut.address);
    const pool = new ethers.Contract(
        poolAddress,
        [
            "function getReserves() view returns(uint112 reserve0, uint112 reserve1, uint32)",
            "function balanceOf(address) view returns(uint)",
        ],
        account
    );
    ```

5. Check Liquidity Token balance and pool reserves.

    ```js
    // Check Liquidity Balance
    const balance = await pool.balanceOf(account.address);
    console.log("Account: Liquidity=", balance.toString());
    // Check Pool reserves
        const { reserve0, reserve1 } = await pool.getReserves();
        console.log(
            "DAI Reserves:",
            DAI_ADDRESS < TUT_ADDRESS ? reserve0.toString() : reserve1.toString()
        );
        console.log(
            "TUT Reserves:",
            TUT_ADDRESS < DAI_ADDRESS ? reserve0.toString() : reserve1.toString()
        );
    })();
    ```

6. Run the script

    ```sh
    node scripts/check-balance.js
    ```
